In [1]:
%matplotlib inline 
#%load_ext autoreload 
%reload_ext autoreload
%autoreload 2

from __future__ import print_function, \
    division, \
    absolute_import
import os

import sep
import h5py

import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
#plt.rc('text', usetex=True)

from astropy.io import fits
from astropy.modeling import models, fitting
import sys
sys.path.append('/Users/RAJ/kungpao')
from kungpao.galsbp import galSBP
from kungpao.display import display_single, random_cmap

Created TAP+ (v1.0) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: False
	Port: 80
	SSL Port: 443


In [2]:
x_images = '/Users/RAJ/anaconda/envs/illustris_profiles/iraf/bin.macosx/x_images.e'
x_ttools = '/Users/RAJ/anaconda/envs/illustris_profiles/iraf_extern/tables/bin.macosx/x_ttools.e'
x_isophote = '/Users/RAJ/anaconda/envs/illustris_profiles/iraf_extern/stsdas/bin.macosx/x_isophote.e'

IMG_CMAP = plt.get_cmap('viridis')
IMG_CMAP.set_bad(color='black')

SEG_CMAP = random_cmap(ncolors=512, background_color=u'white')
SEG_CMAP.set_bad(color='white')
SEG_CMAP.set_under(color='white')

In [3]:
from pyraf import iraf

iraf.tables()   
iraf.stsdas()
iraf.analysis()
iraf.isophote()

iraf.unlearn('ellipse')

iraf.unlearn('bmodel')



      +------------------------------------------------------------+
      |             Space Telescope Tables Package                 |    
      |                  TABLES Version 3.17                       |
      |                                                            |
      |   Space Telescope Science Institute, Baltimore, Maryland   |
      |   Copyright (C) 2014 Association of Universities for       |
      |            Research in Astronomy, Inc.(AURA)               |
      |       See stsdas$copyright.stsdas for terms of use.        |
      |         For help, send e-mail to help@stsci.edu            |
      +------------------------------------------------------------+
tables/:
 fitsio/        tbplot/         tobsolete/      ttools/


      +------------------------------------------------------------+
      |       Space Telescope Science Data Analysis System         |    
      |                   STSDAS Version 3.17                      |
      |                   

In [4]:
def load_pkl(filename):
    try:
        import cPickle as pickle
    except:
        warnings.warn("## cPickle is not available!!")
        import pickle
    
    if os.path.isfile(filename):
        pklFile = open(filename, 'rb')
        data = pickle.load(pklFile)    
        pklFile.close()
    
        return data
    else: 
        warnings.warn("## Can not find %s, return None" % filename)
        return None
    
def save_to_fits(image, name):
    """
    Save a 2-D array as fits image.
    """
    hdu = fits.PrimaryHDU(image)
    hdulist = fits.HDUList([hdu])
    hdulist.writeto(name, overwrite=True)
    
    return

def show_illustris(img_cen, img_sat, img_icl, img_all):
    """
    Show the images of Illustris simulated galaxy.
    """
    fig = plt.figure(figsize=(12, 12))
    fig.subplots_adjust(hspace=0.0, wspace=0.0, 
                        top=1.0, right=1.0, 
                        left=0.0, bottom=0.0)

    ax1 = plt.subplot(2, 2, 1)
    ax1 = display_single(img_all, ax=ax1, 
                         contrast=0.15,
                         scale_bar_length=20.0,
                         scale_bar_loc='right',
                         stretch='log10',
                         pixel_scale=1.0,
                         physical_scale=3.0,
                         color_bar=True)

    ax2 = plt.subplot(2, 2, 2)
    ax2 = display_single(img_cen, ax=ax2, 
                         contrast=0.10,
                         scale_bar_length=20.0,
                         scale_bar_loc='right',
                         stretch='log10',
                         pixel_scale=1.0,
                         physical_scale=3.0,
                         color_bar=True)

    ax3 = plt.subplot(2, 2, 3)
    ax3 = display_single(img_sat, ax=ax3, 
                         contrast=0.12,
                         scale_bar_length=20.0,
                         scale_bar_loc='right',
                         stretch='log10',
                         pixel_scale=1.0,
                         physical_scale=3.0,
                         color_bar=True)

    ax4 = plt.subplot(2, 2, 4)
    ax4 = display_single(img_icl, ax=ax4, 
                         contrast=0.01,
                         scale_bar_length=20.0,
                         scale_bar_loc='right',
                         stretch='log10',
                         pixel_scale=1.0,
                         physical_scale=3.0,
                         color_bar=True)
    
    return fig


In [5]:

n = int(raw_input("How many galaxies do you want to model"))

illustris_file = 'galaxies_orig_11.2.hdf5'

f = h5py.File(illustris_file, 'r')
map_stars_insitu = np.array(f['map_stars_insitu'])
map_size = f.attrs['stellar_map_size']
n_pixels = f.attrs['stellar_map_np']
pixel_scale = 2 * (map_size/n_pixels)

def insitu_list():
    
    fits_list = []
    nonfits_list = []
    fits_prefix = []
    #loc = '/Users/raj/work'
    loc = '.'
    
    for i, insitu in enumerate(map_stars_insitu[:n-1, 0, 1]):
        
        nonfits_list.append(insitu)
        fits_output = 'illustris_xy_%s_insitu.fits' % str(i+1).strip()
        
        fits_prefix.append(fits_output)
        actual_fits = save_to_fits(insitu, fits_output)
        fits_list.append(actual_fits)
        
    
    return nonfits_list, fits_list, fits_prefix
hdf5_list, images_list, fits_prefixlist = insitu_list()





How many galaxies do you want to model3


In [9]:
def onedmodel(data,data1,data2): 
    bkg_cen = sep.Background(data, 
                         bw=10, bh=10, 
                         fw=5, fh=5)
    objs_cen, seg_cen = sep.extract(data - bkg_cen.back(), 
                                20.0,
                                err=bkg_cen.globalrms,
                                minarea=1000,
                                deblend_nthresh=24, 
                                deblend_cont=0.1,
                                segmentation_map=True)
    
    iso_cen_1, iso_cen_1_bin = galSBP.galSBP(data1, 
                                         galX=objs_cen[0]['x'], 
                                         galY=objs_cen[0]['y'], 
                                         galQ=(objs_cen[0]['b'] / objs_cen[0]['a']), 
                                         galPA=(objs_cen[0]['theta'] * 180.0 / np.pi), 
                                         maxSma=150, 
                                         iniSma=10.0, 
                                         stage=1, 
                                         intMode='median', 
                                         ellipStep=0.1,
                                         pix=pixel_scale, 
                                         zpPhoto=0.0,
                                         isophote=x_isophote, 
                                         xttools=x_ttools,
                                         savePng=False, verbose=True)
    try:
        os.remove(parent=data2 + '_ins_ellip_1.fits')
    except Exception:
        pass
    iraf.bmodel(parent=data2 + '_ins.fits', 
            table=iso_cen_1_bin,
            output=data2 + '_ins_ellip_1.fits',
            minsma=0.0,
            highar='no')
    
    iso_cen_2, iso_cen_2_bin = galSBP.galSBP(data1, 
                                         galX=100.0, 
                                         galY=100.0, 
                                         galQ=0.6, 
                                         galPA=-50.0, 
                                         maxSma=220, 
                                         iniSma=10.0, 
                                         stage=2, 
                                         intMode='median', 
                                         ellipStep=0.05,
                                         pix=pixel_scale, 
                                         zpPhoto=0.0,
                                         harmonics='1 2 3 4',
                                         isophote=x_isophote, 
                                         xttools=x_ttools,
                                         recenter=False,
                                         savePng=False, 
                                         verbose=True)
    try:
        os.remove(data2 + '_cen_ellip_2.fits')
    except Exception:
        pass
    
    iraf.bmodel(parent=data2 + '_cen.fits', 
            table=iso_cen_2_bin,
            output=data2 + '_cen_ellip_2.fits',
            minsma=0.0,
            highar='no')
    
    iso_cen_3, iso_cen_3_bin = galSBP.galSBP(data1, 
                                         galX=100.0, 
                                         galY=100.0, 
                                         galQ=iso_cen_2['avg_q'][0], 
                                         galPA=iso_cen_2['avg_pa'][0], 
                                         maxSma=250, 
                                         iniSma=50.0, 
                                         stage=3, 
                                         intMode='median', 
                                         ellipStep=0.05,
                                         pix=pixel_scale, 
                                         zpPhoto=0.0,
                                         harmonics='1 2 3 4',
                                         isophote=x_isophote, 
                                         xttools=x_ttools,
                                         recenter=True,
                                         savePng=False, 
                                         verbose=True)
    try:
        os.remove(data2 + '_cen_ellip_3.fits')
    except Exception:
        pass
    
    iraf.bmodel(parent=data2 + '_cen.fits', 
            table=iso_cen_3_bin,
            output=data2 + '_cen_ellip_3.fits',
            minsma=0.0,
            highar='no')
    
    fig = plt.figure(figsize=(8, 7))
    ax1 = plt.subplot(1, 1, 1)
    ax1.grid(linewidth=2.0, linestyle='--', alpha=0.5)
    dev = models.Sersic1D(amplitude=np.median(iso_cen_3['intens'] / pixel_scale**2), 
                      r_eff=8, n=4.0)
    ax1.plot((iso_cen_3['sma'] * pixel_scale) ** 0.25, 
         np.log10(iso_cen_3['intens'] / pixel_scale**2), linewidth=3.0)
    ax1.plot((iso_cen_3['sma'] * pixel_scale) ** 0.25, 
         np.log10(dev(iso_cen_3['sma'])), linewidth=3.0, alpha=0.5,
         linestyle='--')
    ax1.set_ylabel(r'$\mu_{\star}\ (M_{\star}/\mathrm{kpc}^2)$', fontsize=20)
    ax1.axvline(100.0 ** 0.25, linestyle='--', linewidth=3.0, alpha=0.6)
    ax1.axvline(6.0 ** 0.25, linestyle='-', linewidth=3.0, alpha=0.6, c='r')
    ax1.set_xlim(1.0, 4.5)
    ax1.set_ylim(4.0, 10.0)
    ax1.set_xlabel(r'$\mathrm{SMA/kpc}^{1/4}$', fontsize=25)

_1_D_plot = map(onedmodel,hdf5_list,images_list,fits_prefixlist)

    
    







    
   



TypeError: coercing to Unicode: need string or buffer, NoneType found